In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import pickle
from sklearn import svm, neighbors
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

In [13]:
def process_data_for_labels(ticker):
    hm_days = 7 # how many days
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range (1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

In [14]:
tickers, df = process_data_for_labels('XOM')
df.head()

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,ZBH,ZION,ZTS,XOM_1d,XOM_2d,XOM_3d,XOM_4d,XOM_5d,XOM_6d,XOM_7d
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,26.346825,6.592210,0.0,18.25000,0.0,1.235628,16.274673,15.500,0.0,28.582567,...,0.0,41.202934,0.0,-0.019154,0.034317,0.087789,0.084597,0.069433,0.072626,0.066241
2000-01-04,25.299936,6.403859,0.0,17.81250,0.0,1.198042,14.909401,14.625,0.0,27.449125,...,0.0,39.207741,0.0,0.054516,0.109032,0.105777,0.090317,0.093572,0.087063,0.112287
2000-01-05,26.032759,6.392088,0.0,18.00000,0.0,1.202740,15.204173,15.000,0.0,27.744810,...,0.0,39.161343,0.0,0.051698,0.048611,0.033951,0.037038,0.030865,0.054784,0.033951
2000-01-06,28.126556,6.615753,0.0,18.03125,0.0,1.179249,15.328290,16.000,0.0,27.966566,...,0.0,39.718132,0.0,-0.002935,-0.016875,-0.013939,-0.019809,0.002935,-0.016875,-0.005136
2000-01-07,28.684900,6.686383,0.0,17.93750,0.0,1.212137,16.072983,16.250,0.0,28.508648,...,0.0,39.810936,0.0,-0.013981,-0.011037,-0.016924,0.005887,-0.013981,-0.002208,0.007358


In [26]:
df.iloc[0]['XOM'], df.iloc[1]['XOM']

(21.549222946166992, 21.1364688873291)

In [33]:
print((df.iloc[1]['XOM'] - df.iloc[0]['XOM']) / df.iloc[0]['XOM'])
print((df.iloc[2]['XOM'] - df.iloc[0]['XOM']) / df.iloc[0]['XOM'])
print((df.iloc[3]['XOM'] - df.iloc[0]['XOM']) / df.iloc[0]['XOM'])

-0.019154011254559324
0.03431749285869134
0.08778935101693672


In [ ]:
for i in range (1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

In [23]:
for i in range(1,8):
   diff = (df.iloc[-i]['XOM'] - df.iloc[0]['XOM']) / df.iloc[0]['XOM']
   print(diff)

2.1923373333631906
2.1786127791301806
2.197369374877799
2.208349018264207
2.2033166227046044
2.2156688985368116
2.199657213635795


In [39]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.025
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
           return -1
    return 0

In [40]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)
    df['{}_target'.format(ticker)] = list(map(buy_sell_hold,
                                                df['{}_1d'.format(ticker)],
                                                df['{}_2d'.format(ticker)],
                                                df['{}_3d'.format(ticker)],
                                                df['{}_4d'.format(ticker)],
                                                df['{}_5d'.format(ticker)],
                                                df['{}_6d'.format(ticker)],
                                                df['{}_7d'.format(ticker)],
                                                ))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread: ', Counter(str_vals))
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X = df_vals.values
    y = df['{}_target'.format(ticker)].values

    return X, y, df

In [41]:
X, t, df = extract_featuresets('XOM')

Data spread:  Counter({'0': 2049, '1': 1585, '-1': 1397})


In [42]:
df.head()

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,ZION,ZTS,XOM_1d,XOM_2d,XOM_3d,XOM_4d,XOM_5d,XOM_6d,XOM_7d,XOM_target
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,26.346825,6.592210,0.0,18.25000,0.0,1.235628,16.274673,15.500,0.0,28.582567,...,41.202934,0.0,-0.019154,0.034317,0.087789,0.084597,0.069433,0.072626,0.066241,1
2000-01-04,25.299936,6.403859,0.0,17.81250,0.0,1.198042,14.909401,14.625,0.0,27.449125,...,39.207741,0.0,0.054516,0.109032,0.105777,0.090317,0.093572,0.087063,0.112287,1
2000-01-05,26.032759,6.392088,0.0,18.00000,0.0,1.202740,15.204173,15.000,0.0,27.744810,...,39.161343,0.0,0.051698,0.048611,0.033951,0.037038,0.030865,0.054784,0.033951,1
2000-01-06,28.126556,6.615753,0.0,18.03125,0.0,1.179249,15.328290,16.000,0.0,27.966566,...,39.718132,0.0,-0.002935,-0.016875,-0.013939,-0.019809,0.002935,-0.016875,-0.005136,0
2000-01-07,28.684900,6.686383,0.0,17.93750,0.0,1.212137,16.072983,16.250,0.0,28.508648,...,39.810936,0.0,-0.013981,-0.011037,-0.016924,0.005887,-0.013981,-0.002208,0.007358,0


### Machine learning

Scegliamo il ticker su cui lavorare

In [43]:
ticker = 'XOM'

In [44]:
X, y, df = extract_featuresets(ticker)

Data spread:  Counter({'0': 2049, '1': 1585, '-1': 1397})


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [46]:
# clf = neighbors.KNeighborsClassifier()

clf = VotingClassifier([
    ('lsvc', svm.LinearSVC()),
    ('knn', neighbors.KNeighborsClassifier()),
    ('rfor', RandomForestClassifier())
])

In [47]:
clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lsvc',
                              LinearSVC(C=1.0, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=1000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0)),
                             ('knn',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=5,
                                                   p...
                                                     criterion='gini',
           

Avendo 3 classi, il random guessing corrisponde ad un accuracy del 33%

In [48]:
accuracy = clf.score(X_test, y_test)
accuracy

0.4260731319554849

In [49]:
predictions = clf.predict(X_test)

'Predicted spread: ', Counter(predictions) 

('Predicted spread: ', Counter({1: 197, 0: 853, -1: 208}))